In [1]:
import pandas as pd
import numpy as np

c:\Users\Atfan\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
df = pd.read_csv("../Input Data/r001_new.output_trips.csv", delimiter= ";")

In [7]:
df.head()

,person,trip_number,trip_id,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,longest_distance_mode,...,start_facility_id,start_link,start_x,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop
0,container_100_106_165_0,1,container_100_106_165_0_1,15:30:59,02:50:09,00:00:00,284514,217765,NaN,car,...,NaN,14006,2.716909e+06,1.078734e+06,NaN,169866,2611207.0,1269126.0,NaN,NaN
1,container_100_106_166_1,1,container_100_106_166_1_1,34:06:10,02:47:33,00:00:00,284486,217705,NaN,car,...,NaN,14006,2.716909e+06,1.078734e+06,NaN,65715,2611193.0,1269049.0,NaN,NaN
2,container_100_47_163_0,1,container_100_47_163_0_1,13:34:54,02:13:54,00:00:00,235173,156401,NaN,car,...,NaN,60888,2.582596e+06,1.114786e+06,NaN,112307,2613992.0,1268004.0,NaN,NaN
3,container_100_47_164_1,1,container_100_47_164_1_1,44:04:54,10:05:44,00:00:00,243885,174083,NaN,car,...,NaN,26859,2.577384e+06,1.101500e+06,NaN,81135,2616131.0,1271216.0,NaN,NaN
4,container_100_47_88_2,1,container_100_47_88_2_1,57:38:13,02:44:45,00:00:00,254946,170191,NaN,car,...,NaN,123449,2.582917e+06,1.104633e+06,NaN,161755,2615755.0,1271626.0,NaN,NaN


In [8]:
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------
TERMINALS_FILE = "../Input Data/terminals.csv"          # path to your terminals.csv
TRIPS_FILE     = "../Input Data/r001_new.output_trips.csv"  # path to your MATSim trips csv
SEP = ";"                                 # both files are semicolon-separated

WEIGHT_COLUMN = None

# MATSim scenario is typically 1 representative day
DAILY_TO_WEEKLY_FACTOR = 7


# -------------------------------------------------------------------
# 1. Load data
# -------------------------------------------------------------------
terminals = pd.read_csv(TERMINALS_FILE, sep=SEP, engine="python", on_bad_lines="skip")
trips = pd.read_csv(TRIPS_FILE, sep=SEP, engine="python", on_bad_lines="skip")

# Terminals must have: short_name, x, y
required_term_cols = {"short_name", "x", "y"}
if not required_term_cols.issubset(terminals.columns):
    missing = required_term_cols - set(terminals.columns)
    raise ValueError(f"terminals.csv missing columns: {missing}")

# Trips must have: start_x, start_y, end_x, end_y, trav_time, traveled_distance
required_trip_cols = {"start_x", "start_y", "end_x", "end_y", "trav_time", "traveled_distance"}
if not required_trip_cols.issubset(trips.columns):
    missing = required_trip_cols - set(trips.columns)
    raise ValueError(f"trips file missing columns: {missing}")

# Prepare terminal table
terms = terminals[["short_name", "x", "y"]].copy()
terms.rename(columns={"short_name": "terminal"}, inplace=True)

# -------------------------------------------------------------------
# 2. Assign each trip to nearest origin & destination terminal
# -------------------------------------------------------------------
term_xy = terms[["x", "y"]].to_numpy()                       # (T, 2)
start_xy = trips[["start_x", "start_y"]].to_numpy()          # (N, 2)
end_xy   = trips[["end_x", "end_y"]].to_numpy()              # (N, 2)

# origin: squared distance (N,T)
diff_start = start_xy[:, None, :] - term_xy[None, :, :]
dist2_start = (diff_start ** 2).sum(axis=2)
origin_idx = dist2_start.argmin(axis=1)
origin_term = terms.iloc[origin_idx]["terminal"].values

# destination: squared distance (N,T)
diff_end = end_xy[:, None, :] - term_xy[None, :, :]
dist2_end = (diff_end ** 2).sum(axis=2)
dest_idx = dist2_end.argmin(axis=1)
dest_term = terms.iloc[dest_idx]["terminal"].values

# Attach to trips DataFrame
trips_with_terms = trips.copy()
trips_with_terms["origin_terminal"] = origin_term
trips_with_terms["destination_terminal"] = dest_term

# -------------------------------------------------------------------
# 3. Parse travel time and distance
# -------------------------------------------------------------------
def time_to_hours(t):
    """
    Convert 'hh:mm:ss' string to hours (float).
    Returns NaN if parsing fails.
    """
    try:
        h, m, s = str(t).split(":")
        return int(h) + int(m) / 60.0 + int(s) / 3600.0
    except Exception:
        return np.nan

trips_with_terms["trav_time_h"] = trips_with_terms["trav_time"].apply(time_to_hours)
trips_with_terms["distance_km"] = trips_with_terms["traveled_distance"] / 1000.0

# Weight (for exposure)
if WEIGHT_COLUMN is not None and WEIGHT_COLUMN in trips_with_terms.columns:
    trips_with_terms["weight"] = trips_with_terms[WEIGHT_COLUMN]
else:
    trips_with_terms["weight"] = 1.0  # each trip = 1 TEU


# -------------------------------------------------------------------
# 4. Aggregate to OD-level stats
# -------------------------------------------------------------------
od_stats = (
    trips_with_terms
    .groupby(["origin_terminal", "destination_terminal"], as_index=False)
    .agg(
        daily_TEU=("weight", "sum"),
        mean_trav_time_h=("trav_time_h", "mean"),
        median_trav_time_h=("trav_time_h", "median"),
        mean_distance_km=("distance_km", "mean"),
        median_distance_km=("distance_km", "median"),
    )
)

od_stats["weekly_TEU"] = od_stats["daily_TEU"] * DAILY_TO_WEEKLY_FACTOR

# Sort for readability
od_stats.sort_values(["origin_terminal", "destination_terminal"], inplace=True)

# -------------------------------------------------------------------
# 5. Inspect or export
# -------------------------------------------------------------------
print("OD-level road exposure and performance stats:")
print(od_stats)

OD-level road exposure and performance stats:
   origin_terminal destination_terminal  daily_TEU  mean_trav_time_h  \
0             AARA                 BERN        1.0          1.001389   
1             AARA                 FRIB        1.0          1.218889   
2             AARA                 LAUS        3.0          1.739352   
3             AARA                 STAB        2.0          2.271528   
4             AARA                 VISP        3.0          3.223333   
5             AARA                 WINT        3.0          1.306019   
6             BASE                 STAB      252.0          3.147622   
7             BERN                 VISP        1.0          9.560000   
8             LAUS                 AARA        2.0          1.789444   
9             LUZE                 LAUS        1.0          2.457778   
10            LUZE                 STAB        1.0          2.326111   
11            OLTE                 STAB        2.0          2.364167   
12            SCHA

In [9]:
od_stats.to_excel("../model_output/OD_STATS_new.xlsx")

In [ ]:
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------
TERMINALS_FILE = "../Input Data/terminals.csv"          # path to your terminals.csv
TRIPS_FILE     = "../Input Data/r001.output_trips.csv"  # path to your MATSim trips csv
SEP = ";"                                 # both files are semicolon-separated

WEIGHT_COLUMN = None

# MATSim scenario is typically 1 representative day
DAILY_TO_WEEKLY_FACTOR = 7


# -------------------------------------------------------------------
# 1. Load data
# -------------------------------------------------------------------
terminals = pd.read_csv(TERMINALS_FILE, sep=SEP, engine="python", on_bad_lines="skip")
trips = pd.read_csv(TRIPS_FILE, sep=SEP, engine="python", on_bad_lines="skip")

# Terminals must have: short_name, x, y
required_term_cols = {"short_name", "x", "y"}
if not required_term_cols.issubset(terminals.columns):
    missing = required_term_cols - set(terminals.columns)
    raise ValueError(f"terminals.csv missing columns: {missing}")

# Trips must have at least these columns
required_trip_cols = {
    "start_x", "start_y",
    "end_x", "end_y",
    "trav_time",
    "traveled_distance",
    "dep_time",          # <-- NEW: we will use departure time
}
if not required_trip_cols.issubset(trips.columns):
    missing = required_trip_cols - set(trips.columns)
    raise ValueError(f"trips file missing columns: {missing}")

# Prepare terminal table
terms = terminals[["short_name", "x", "y"]].copy()
terms.rename(columns={"short_name": "terminal"}, inplace=True)


# -------------------------------------------------------------------
# 2. Assign each trip to nearest origin & destination terminal
# -------------------------------------------------------------------
term_xy = terms[["x", "y"]].to_numpy()                       # (T, 2)
start_xy = trips[["start_x", "start_y"]].to_numpy()          # (N, 2)
end_xy   = trips[["end_x", "end_y"]].to_numpy()              # (N, 2)

# origin: squared distance (N,T)
diff_start = start_xy[:, None, :] - term_xy[None, :, :]
dist2_start = (diff_start ** 2).sum(axis=2)
origin_idx = dist2_start.argmin(axis=1)
origin_term = terms.iloc[origin_idx]["terminal"].values

# destination: squared distance (N,T)
diff_end = end_xy[:, None, :] - term_xy[None, :, :]
dist2_end = (diff_end ** 2).sum(axis=2)
dest_idx = dist2_end.argmin(axis=1)
dest_term = terms.iloc[dest_idx]["terminal"].values

# Attach to trips DataFrame
trips_with_terms = trips.copy()
trips_with_terms["origin_terminal"] = origin_term
trips_with_terms["destination_terminal"] = dest_term


# -------------------------------------------------------------------
# 3. Parse times and distances
# -------------------------------------------------------------------
def time_to_hours(t):
    """
    Convert 'hh:mm:ss' string (possibly >24h, e.g. '34:06:10') to hours (float).
    Returns NaN if parsing fails.
    """
    try:
        h, m, s = str(t).split(":")
        return int(h) + int(m) / 60.0 + int(s) / 3600.0
    except Exception:
        return np.nan

# Travel time in hours
trips_with_terms["trav_time_h"] = trips_with_terms["trav_time"].apply(time_to_hours)

# Departure time in hours since simulation start
trips_with_terms["dep_time_h"] = trips_with_terms["dep_time"].apply(time_to_hours)

# Arrival time in hours since simulation start
trips_with_terms["arrival_time_h"] = (
    trips_with_terms["dep_time_h"] + trips_with_terms["trav_time_h"]
)

# Optional: fold into hour-of-day [0,24)
trips_with_terms["dep_hour_of_day"] = trips_with_terms["dep_time_h"] % 24.0
trips_with_terms["arrival_hour_of_day"] = trips_with_terms["arrival_time_h"] % 24.0

# Distance in km
trips_with_terms["distance_km"] = trips_with_terms["traveled_distance"] / 1000.0

# Weight (for exposure)
if WEIGHT_COLUMN is not None and WEIGHT_COLUMN in trips_with_terms.columns:
    trips_with_terms["weight"] = trips_with_terms[WEIGHT_COLUMN]
else:
    trips_with_terms["weight"] = 1.0  # each trip = 1 TEU


# -------------------------------------------------------------------
# 4. Aggregate to OD-level stats (now including timing)
# -------------------------------------------------------------------
od_stats = (
    trips_with_terms
    .groupby(["origin_terminal", "destination_terminal"], as_index=False)
    .agg(
        daily_TEU          = ("weight", "sum"),
        mean_trav_time_h   = ("trav_time_h", "mean"),
        median_trav_time_h = ("trav_time_h", "median"),
        mean_distance_km   = ("distance_km", "mean"),
        median_distance_km = ("distance_km", "median"),

        # Optional: hour-of-day summaries (folded into 0–24)
        mean_dep_hour_of_day       = ("dep_hour_of_day", "mean"),
        mean_arrival_hour_of_day   = ("arrival_hour_of_day", "mean"),
    )
)

od_stats["weekly_TEU"] = od_stats["daily_TEU"] * DAILY_TO_WEEKLY_FACTOR

# Sort for readability
od_stats.sort_values(["origin_terminal", "destination_terminal"], inplace=True)

# -------------------------------------------------------------------
# 5. Inspect or export
# -------------------------------------------------------------------
print("OD-level road exposure and performance stats (with timing):")
print(od_stats.head())

# You probably want to export this for the master & TS:
od_stats.to_excel("../Input Data/OD_STATS_with_times.xlsx", index=False)


OD-level road exposure and performance stats (with timing):
  origin_terminal destination_terminal  daily_TEU  mean_trav_time_h  \
0            AARA                 BERN        1.0          1.001389   
1            AARA                 FRIB        1.0          1.218889   
2            AARA                 LAUS        3.0          1.739352   
3            AARA                 STAB        2.0          2.271528   
4            AARA                 VISP        3.0          3.223333   

   median_trav_time_h  mean_distance_km  median_distance_km  mean_dep_time_h  \
0            1.001389         93.682000             93.6820        35.949444   
1            1.218889        128.465000            128.4650        35.662778   
2            1.773611        156.277333            153.8680        28.712037   
3            2.271528        206.895500            206.8955        23.179167   
4            3.292500        199.856333            195.2850        33.054259   

   median_dep_time_h  mean_arriv